In [2]:
# pip install transformers tqdm boto3 requests regex -q

In [3]:
import os
import pickle 
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt 
from datetime import datetime
import random
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from IPython.display import clear_output
from torch.nn.utils.rnn import pad_sequence
from numpy.linalg import norm

import warnings
warnings.filterwarnings("ignore")

# 準備文本數據

In [4]:
# News
News = pd.read_pickle("../corpus/News/UnfilterNewsData_1222.pkl")
News.rename(columns = {'pub_date':'Publication_Time', 'title':'Post_Title'}, inplace = True)
News['Publication_Date'] = News['Publication_Time'].apply(lambda x: str(x.date()))
News = News.drop(columns=['content','search_key', 'link', 'search_time', 'description','source','Publication_Time'])
News['Publication_Date']= pd.to_datetime(News['Publication_Date'])
News['Length'] = News['Post_Title'].apply(lambda x: len(x))
News

,Post_Title,Publication_Date,Length
0,【科技周報】鴻海入股紫光破局？台積總裁魏哲家打臉「去台化」、王瑞瑜打造台塑能源王國… - Y...,2022-12-23,54
1,鴻海Model T廠區接駁電動巴士大公開 員工往返廠區方便喊讚 - ETtoday財經雲,2022-12-22,44
2,影/鴻海電巴MODEL T土城現蹤 竟是員工接博車 - 經濟日報,2022-12-23,32
3,鴻海線上嘉年華 三大亮點 - Yahoo奇摩新聞,2022-12-22,24
4,國巨、鴻海的「國創半導體」成軍一年半 近況怎樣？ - 經濟日報,2022-12-22,31
...,...,...,...
23069,11月定投熱門金融股出爐5檔殖利率逾5％ - 工商時報,2022-12-15,27
23070,中環旗下中嘉賣陽明賺681萬 投資報酬率2.1%、同期大盤漲6% - ETtoday財經雲,2022-12-13,45
23071,退休金靠自己 年輕人投資ETF大增至五成 - Yahoo奇摩新聞,2022-12-14,32
23072,定存5千萬即可免工作？網看笑：買房、0056、00878都爽贏 - 聯合新聞網,2022-12-12,39


In [5]:
# Facebook
fb_data = pd.read_pickle("../corpus/Facebook_Post/UnfilterFaceBookData_1222.pkl")
fb_data['Publication_Date'] = fb_data['Publication_Time'].apply(lambda x: str(x.date()))
fb_data = fb_data.drop(columns=['Publisher', 'Post_url', 'Likes',
       'Reactions', 'Comments', 'Shares', 'Cover_photo','Publication_Time'])
fb_data['Publication_Date']= pd.to_datetime(fb_data['Publication_Date'])
fb_data.rename(columns = {'Post_Content':'Post_Title'}, inplace = True)
fb_data['Length'] = fb_data['Post_Title'].apply(lambda x: len(x))
fb_data

,Post_Title,Publication_Date,Length
0,澳洲外長黃英賢藉著中澳建交50週年的契機，前往北京與中國外長王毅會晤。中國媒體以修補兩國關係...,2022-12-24,63
1,當前世界各國紛紛升息抑制物價，土耳其卻反其道而行，連續降息導致貨幣貶值，通膨連月飆升到80%...,2022-12-24,50
2,台北市立委補選進入倒數，民進黨候選人吳怡農跟國民黨王鴻薇，上午不約而同到松山的草埔市場掃街拜...,2022-12-24,59
3,一名李小姐在網路商城「博客來」擔任清潔員，指出服務20多年照常打卡上下班，也符合主管指示工作...,2022-12-24,97
4,新一波疫情快速升溫！昨（24）日新增1萬8千多例新冠本土病例，已連續兩天增幅超過2成。，,2022-12-24,44
...,...,...,...
8829,【Yahoo論壇／商業周刊】，巨大今年前3季淨利突破56億元，年增率達16％，創下歷史新高水...,2022-12-15,186
8830,地震好搖⋯你有收到國家級警報嗎,2022-12-15,15
8831,美媒表示，此舉是為了阻止它們購買某些美國零組件。，,2022-12-15,25
8832,范瑋琪： 即便萬般不捨，您已卸下勞苦重擔，安息主懷，,2022-12-15,26


In [6]:
Corpus = pd.concat([fb_data, News])
Corpus

,Post_Title,Publication_Date,Length
0,澳洲外長黃英賢藉著中澳建交50週年的契機，前往北京與中國外長王毅會晤。中國媒體以修補兩國關係...,2022-12-24,63
1,當前世界各國紛紛升息抑制物價，土耳其卻反其道而行，連續降息導致貨幣貶值，通膨連月飆升到80%...,2022-12-24,50
2,台北市立委補選進入倒數，民進黨候選人吳怡農跟國民黨王鴻薇，上午不約而同到松山的草埔市場掃街拜...,2022-12-24,59
3,一名李小姐在網路商城「博客來」擔任清潔員，指出服務20多年照常打卡上下班，也符合主管指示工作...,2022-12-24,97
4,新一波疫情快速升溫！昨（24）日新增1萬8千多例新冠本土病例，已連續兩天增幅超過2成。，,2022-12-24,44
...,...,...,...
23069,11月定投熱門金融股出爐5檔殖利率逾5％ - 工商時報,2022-12-15,27
23070,中環旗下中嘉賣陽明賺681萬 投資報酬率2.1%、同期大盤漲6% - ETtoday財經雲,2022-12-13,45
23071,退休金靠自己 年輕人投資ETF大增至五成 - Yahoo奇摩新聞,2022-12-14,32
23072,定存5千萬即可免工作？網看笑：買房、0056、00878都爽贏 - 聯合新聞網,2022-12-12,39


In [7]:
Corpus['Length'].mean()

55.75222514729848

In [8]:
TOKEN_LENGTH = 40

# BERT + Cosine Similarity

In [9]:
#download pretrained model
Model = 'bert-base-chinese'
tokenizer = AutoTokenizer.from_pretrained(Model)
model = AutoModelForMaskedLM.from_pretrained(Model)
clear_output()

In [10]:
# Create embeddings
def get_embeddings(text,token_length=TOKEN_LENGTH):
    # Create tensor
    word_pieces = ["[CLS]"]
    sep_text = tokenizer.tokenize(text)
    word_pieces += sep_text + ["[SEP]"]
    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    tokens_tensor = torch.tensor(ids)
    return tokens_tensor

In [11]:
# calculate similarity
def Cosine_Similarity(text1,text2):
    emb1 = get_embeddings(text1)
    emb2 = get_embeddings(text2)
    A = pad_sequence([emb1, emb2], batch_first=True)[0].numpy().reshape(-1,)
    B = pad_sequence([emb1, emb2], batch_first=True)[1].numpy().reshape(-1,)
    cosine = np.dot(A,B)/(norm(A)*norm(B))
    return cosine

In [12]:
# Test
sent1 = "鴻海生產重鎮靜態管理、美光1β製程在日量產 今日必看財經新聞"
sent2 = "鴻海富智康前3季虧損擴大3大不利因素影響| 聯合新聞網"
sent3 = "彰化福興鄉的同安村近期空氣中都散發惡臭，也讓民眾懷疑一處農地倉庫偷埋電鍍廢土。"
test_corpus = [sent1, sent2, sent3]

In [13]:
Cosine_Similarity(sent1, sent2)

0.6627511943812088

In [14]:
Cosine_Similarity(sent2, sent3)

0.6585809330928357

In [15]:
Cosine_Similarity(sent1, sent3)

0.7602045768350346

# xmnlp

https://github.com/SeanLee97/xmnlp

In [16]:
import xmnlp
xmnlp.set_model('../xmnlp-onnx-models-v5/xmnlp-onnx-models')

In [17]:
from xmnlp.sv import SentenceVector

In [18]:
Corpus.head()

,Post_Title,Publication_Date,Length
0,澳洲外長黃英賢藉著中澳建交50週年的契機，前往北京與中國外長王毅會晤。中國媒體以修補兩國關係...,2022-12-24,63
1,當前世界各國紛紛升息抑制物價，土耳其卻反其道而行，連續降息導致貨幣貶值，通膨連月飆升到80%...,2022-12-24,50
2,台北市立委補選進入倒數，民進黨候選人吳怡農跟國民黨王鴻薇，上午不約而同到松山的草埔市場掃街拜...,2022-12-24,59
3,一名李小姐在網路商城「博客來」擔任清潔員，指出服務20多年照常打卡上下班，也符合主管指示工作...,2022-12-24,97
4,新一波疫情快速升溫！昨（24）日新增1萬8千多例新冠本土病例，已連續兩天增幅超過2成。，,2022-12-24,44


In [19]:
# Test
query = '鴻海攜沙國公共投資基金合資國家電動車品牌 進軍中東市場 - Anue鉅亨'
docs = [
    '鴻海鄭州廠稼動率維持七成 Q4 iPhone出貨調降300萬支 - 聯合新聞網',
    ' 教育盃國民中小學創意環保科學趣味競賽 智高城堡攻防戰 - Yahoo奇摩新聞'
]

sv = SentenceVector(genre='通用')
for doc in docs:
    print('doc:', doc)
    print('similarity:', sv.similarity(query, doc))
print('most similar doc:', sv.most_similar(query, docs))
print('query representation shape:', sv.transform(query).shape)

doc: 鴻海鄭州廠稼動率維持七成 Q4 iPhone出貨調降300萬支 - 聯合新聞網
similarity: 0.8467112
doc:  教育盃國民中小學創意環保科學趣味競賽 智高城堡攻防戰 - Yahoo奇摩新聞
similarity: 0.50202054
most similar doc: [('鴻海鄭州廠稼動率維持七成 Q4 iPhone出貨調降300萬支 - 聯合新聞網', 11.606600327569268)]
query representation shape: (312,)


In [20]:
# 人工挑20篇最符合的文章作為query key
query_sents=['逢高倒貨？八大官股券商連3天賣超台股 台積電遭狂賣10億',
            '聯電ADR2日下跌0.10美元跌幅1.67%折台股37.90元',
            '勞動基金前9月虧損5124億 台積電、國泰金10月股價仍下挫 全年恐難轉正',
            '台股低量反彈161點 三大法人買超逾75億元',
            '台積電領跌 台股蒙「鷹」影跌113點收12986點 失守月線',
            '群光第三季獲利創歷史次高 全年挑戰賺一股本',
            '盤中速報- 研華(2395)股價急跌至277.0元，跌幅達7.05%',
            '中國堅持清零及Fed鷹派言論，恆指週四重挫487.68點或3.08%，類股齊黑',
            '京城銀前10月獲利年減八成，EPS 1.12元，手續費淨收益年增23%',
            '蘋概股給力開高震盪站回10日線 一度逼近萬三',
            '慧洋前10月稅前淨利破百億 樂觀2023散裝需求回穩',
            '盤中速報 - 玻璃陶瓷類指數類股表現疲軟，跌幅2.26%，總成交額1.05億',
            '電子權值股不振台股早盤一度跌逾百點',
            'NB代工大廠10月出貨較9月下滑，廣達、英業達伺服器撐盤，雙創同期新高',
            '緯穎獲利灌頂，緯創Q3合併營業利益創高，今股價噴漲',
            '台股大漲291點 外資歸隊狂買241億元 三大法人買超230.55億元',
            ' 工具機雙雄上銀及亞德客-KY第三季獲利一喜一憂，Q4轉趨保守',
            ' 盤中速報- 電機機械類指數類股表現強勁，漲幅2.13%，總成交額37.49億',
            '台積電受巴菲特青睞 市值重返12兆 激勵台股漲371點收14,546點',
            '〈台股盤前要聞〉聯發科將推新晶片、群創下調資本支出']

In [21]:
def Calculate_Similarity(text):
    query = random.choice(query_sents)
    score = sv.similarity(query, text)
    return score

In [22]:
Corpus

,Post_Title,Publication_Date,Length
0,澳洲外長黃英賢藉著中澳建交50週年的契機，前往北京與中國外長王毅會晤。中國媒體以修補兩國關係...,2022-12-24,63
1,當前世界各國紛紛升息抑制物價，土耳其卻反其道而行，連續降息導致貨幣貶值，通膨連月飆升到80%...,2022-12-24,50
2,台北市立委補選進入倒數，民進黨候選人吳怡農跟國民黨王鴻薇，上午不約而同到松山的草埔市場掃街拜...,2022-12-24,59
3,一名李小姐在網路商城「博客來」擔任清潔員，指出服務20多年照常打卡上下班，也符合主管指示工作...,2022-12-24,97
4,新一波疫情快速升溫！昨（24）日新增1萬8千多例新冠本土病例，已連續兩天增幅超過2成。，,2022-12-24,44
...,...,...,...
23069,11月定投熱門金融股出爐5檔殖利率逾5％ - 工商時報,2022-12-15,27
23070,中環旗下中嘉賣陽明賺681萬 投資報酬率2.1%、同期大盤漲6% - ETtoday財經雲,2022-12-13,45
23071,退休金靠自己 年輕人投資ETF大增至五成 - Yahoo奇摩新聞,2022-12-14,32
23072,定存5千萬即可免工作？網看笑：買房、0056、00878都爽贏 - 聯合新聞網,2022-12-12,39


In [24]:
# 大約跑10 mins
Corpus['Similarity'] = Corpus['Post_Title'].apply(lambda x: Calculate_Similarity(x))

In [25]:
Corpus.describe()

,Length,Similarity
count,31908.000000,31908.000000
mean,55.752225,0.499395
std,98.659044,0.157032
min,0.000000,-0.107703
25%,27.000000,0.428961
50%,35.000000,0.523681
75%,45.000000,0.603722
max,4303.000000,0.972292


In [26]:
Corpus

,Post_Title,Publication_Date,Length,Similarity
0,澳洲外長黃英賢藉著中澳建交50週年的契機，前往北京與中國外長王毅會晤。中國媒體以修補兩國關係...,2022-12-24,63,0.347012
1,當前世界各國紛紛升息抑制物價，土耳其卻反其道而行，連續降息導致貨幣貶值，通膨連月飆升到80%...,2022-12-24,50,0.358250
2,台北市立委補選進入倒數，民進黨候選人吳怡農跟國民黨王鴻薇，上午不約而同到松山的草埔市場掃街拜...,2022-12-24,59,0.486277
3,一名李小姐在網路商城「博客來」擔任清潔員，指出服務20多年照常打卡上下班，也符合主管指示工作...,2022-12-24,97,0.540797
4,新一波疫情快速升溫！昨（24）日新增1萬8千多例新冠本土病例，已連續兩天增幅超過2成。，,2022-12-24,44,0.538200
...,...,...,...,...
23069,11月定投熱門金融股出爐5檔殖利率逾5％ - 工商時報,2022-12-15,27,0.485338
23070,中環旗下中嘉賣陽明賺681萬 投資報酬率2.1%、同期大盤漲6% - ETtoday財經雲,2022-12-13,45,0.663146
23071,退休金靠自己 年輕人投資ETF大增至五成 - Yahoo奇摩新聞,2022-12-14,32,0.370890
23072,定存5千萬即可免工作？網看笑：買房、0056、00878都爽贏 - 聯合新聞網,2022-12-12,39,0.621457


In [27]:
similarity_mean = Corpus['Similarity'].mean()
Filtered_Corpus = Corpus.query('Similarity >= @similarity_mean').reset_index(drop=True)
Filtered_Corpus

,Post_Title,Publication_Date,Length,Similarity
0,一名李小姐在網路商城「博客來」擔任清潔員，指出服務20多年照常打卡上下班，也符合主管指示工作...,2022-12-24,97,0.540797
1,新一波疫情快速升溫！昨（24）日新增1萬8千多例新冠本土病例，已連續兩天增幅超過2成。，,2022-12-24,44,0.538200
2,明（25）天就是議長選舉日，但台南前天卻爆出有議員遭恐嚇，要求議長選舉只能「不出席」或「投自...,2022-12-24,72,0.507028
3,寒流急凍，氣象局持續對全台20縣市，發布低溫特報。加上輻射冷卻影響，今（24）晨平地最低溫出...,2022-12-24,62,0.558108
4,展望2023年，歐美媒體都預測了世界局勢可能走向。，,2022-12-24,26,0.611664
...,...,...,...,...
18171,【台股盤後】美通膨數據降溫 大漲216點 - Yahoo奇摩股市,2022-12-14,32,0.688915
18172,3重點篩選賺配息又賺價差 台股高息ETF大盤點 - ETtoday財經雲,2022-12-14,36,0.600608
18173,中環旗下中嘉賣陽明賺681萬 投資報酬率2.1%、同期大盤漲6% - ETtoday財經雲,2022-12-13,45,0.663146
18174,定存5千萬即可免工作？網看笑：買房、0056、00878都爽贏 - 聯合新聞網,2022-12-12,39,0.621457


In [28]:
os.chdir('../corpus')

In [29]:
Filtered_Corpus.to_pickle('Filtered_Corpus_to_1222')
Corpus.to_pickle('Unfiltered_Corpus_to_1222')